In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.7/877.7 kB 13.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir /content/drive/MyDrive/yolonew

In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np
from google.colab.patches import cv2_imshow   # 코렙에서 이미지를 보여주기 위해 필요
from collections import deque  # file 입력과 출력 관련한 자료구조 (사람 카운트 용도)

# YOLOv8 모델 불러오기
model = YOLO('yolov8n.pt')   # yolo 나노 버전 - 비교적 가벼운 모델(80개의 클래스 자동 인식)

# 비디오 캡처 초기화
cap = cv2.VideoCapture("/content/drive/MyDrive/yolonew/newyork.mp4")

# 비디오 저장 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # mp4 형태로 저장
out = cv2.VideoWriter("/content/drive/MyDrive/yolonew/output2_with_counts.mp4", fourcc, 30.0,
                      (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# 카운팅 변수 초기화
total_in_count = 0  # 누적 입장 수
total_out_count = 0  # 누적 퇴장 수
detected_ids = {}   # 객체(사람) id와 좌표를 저장할 딕셔너리
next_id = 0   # 새로운 객체에 부여할 고유 id
exit_buffer = 30  # ID가 삭제되기 전까지 유지되는 프레임 수
id_buffer = deque(maxlen=50)  # 최근 ID 기록용 - 사람 1명당 ID 1개(최대 50개)

while cap.isOpened():  # 비디오가 열려있는 동안 반복
    ret, frame = cap.read()  # 프레임을 하나씩 읽어오기
    if not ret:   # 읽기 성공 여부 - 프레임이 없으면
        break     # 루프 중단시키기

    # 객체 탐지 수행
    results = model(frame)

    current_ids = set()  # 현재 프레임에서 감지된 ID를 저장할 집합

    for det in results[0].boxes:  # 감지된 객체 각각에 대해 반복 처리
        x1, y1, x2, y2 = map(int, det.xyxy[0])  # 박스 좌표
        conf = det.conf[0].item()  # 사람일 확률에 대한 신뢰도
        cls = int(det.cls[0].item())  # 사람 클래스 번호

        if cls == 0 and conf > 0.5:  # 클래스 0은 사람

            # 중심 좌표 계산
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)

            # ID 생성 및 관리
            object_id = None
            for id, (prev_x, prev_y, frames) in detected_ids.items():
                if abs(prev_x - center_x) < 50 and abs(prev_y - center_y) < 50:
                    object_id = id
                    detected_ids[id] = (center_x, center_y, exit_buffer)  # 프레임 수 초기화
                    break

            if object_id is None:
                object_id = next_id
                detected_ids[object_id] = (center_x, center_y, exit_buffer)
                next_id += 1
                total_in_count += 1  # 새 ID 감지 시 입장으로 간주

            current_ids.add(object_id)  # 현재 프레임에 감지된 ID 추가

            # 경계 상자와 ID 표시
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, f'ID: {object_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # 이전 프레임에서 존재했지만 현재 프레임에 없는 ID의 프레임 수 감소
    # 특정 사람이 다른 물체에 가려지거나 했을 떄도 계속 같은 사람임을 나타내주기 위한 코드
    for id in list(detected_ids.keys()):
        if id not in current_ids:
            prev_x, prev_y, frames = detected_ids[id]
            if frames > 0:
                detected_ids[id] = (prev_x, prev_y, frames - 1)
            else:
                del detected_ids[id]
                total_out_count += 1  # ID가 사라진 경우 퇴장으로 간주

    # 누적 카운트 표시
    cv2.putText(frame, f'Total In: {total_in_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'Total Out: {total_out_count}', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # 비디오에 현재 프레임 저장
    out.write(frame)

    # Colab 환경에서 현재 프레임 표시
    # cv2_imshow(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 모든 작업 완료 후 자원 해제
cap.release()
out.release()
cv2.destroyAllWindows()


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 72.1MB/s]



0: 384x640 11 persons, 2 cars, 1 handbag, 357.3ms
Speed: 27.3ms preprocess, 357.3ms inference, 40.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 2 cars, 1 handbag, 158.7ms
Speed: 4.5ms preprocess, 158.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 2 cars, 1 handbag, 192.3ms
Speed: 9.3ms preprocess, 192.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 2 cars, 1 handbag, 158.4ms
Speed: 4.7ms preprocess, 158.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 2 cars, 1 handbag, 142.3ms
Speed: 4.7ms preprocess, 142.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 2 cars, 1 handbag, 164.7ms
Speed: 7.7ms preprocess, 164.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 2 cars, 1 handbag, 158.9ms
Speed: 5.0ms preprocess, 158.9ms inference, 1.3ms pos

In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np
from google.colab.patches import cv2_imshow   # 코렙에서 이미지를 보여주기 위해 필요
from collections import deque  # file 입력과 출력 관련한 자료구조 (사람 카운트 용도)

# YOLOv8 모델 불러오기
model = YOLO('yolov8n.pt')   # yolo 나노 버전 - 비교적 가벼운 모델(80개의 클래스 자동 인식)

# 비디오 캡처 초기화
cap = cv2.VideoCapture("/content/drive/MyDrive/yolonew/videoplayback_exported.mp4")

# 비디오 저장 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # mp4 형태로 저장
out = cv2.VideoWriter("/content/drive/MyDrive/yolonew/output3_with_counts.mp4", fourcc, 30.0,
                      (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# 카운팅 변수 초기화
total_in_count = 0  # 누적 입장 수
total_out_count = 0  # 누적 퇴장 수
detected_ids = {}   # 객체(사람) id와 좌표를 저장할 딕셔너리
next_id = 0   # 새로운 객체에 부여할 고유 id
exit_buffer = 30  # ID가 삭제되기 전까지 유지되는 프레임 수
id_buffer = deque(maxlen=50)  # 최근 ID 기록용 - 사람 1명당 ID 1개(최대 50개)

while cap.isOpened():  # 비디오가 열려있는 동안 반복
    ret, frame = cap.read()  # 프레임을 하나씩 읽어오기
    if not ret:   # 읽기 성공 여부 - 프레임이 없으면
        break     # 루프 중단시키기

    # 객체 탐지 수행
    results = model(frame)

    current_ids = set()  # 현재 프레임에서 감지된 ID를 저장할 집합

    for det in results[0].boxes:  # 감지된 객체 각각에 대해 반복 처리
        x1, y1, x2, y2 = map(int, det.xyxy[0])  # 박스 좌표
        conf = det.conf[0].item()  # 사람일 확률에 대한 신뢰도
        cls = int(det.cls[0].item())  # 사람 클래스 번호

        if cls == 0 and conf > 0.5:  # 클래스 0은 사람

            # 중심 좌표 계산
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)

            # ID 생성 및 관리
            object_id = None
            for id, (prev_x, prev_y, frames) in detected_ids.items():
                if abs(prev_x - center_x) < 50 and abs(prev_y - center_y) < 50:
                    object_id = id
                    detected_ids[id] = (center_x, center_y, exit_buffer)  # 프레임 수 초기화
                    break

            if object_id is None:
                object_id = next_id
                detected_ids[object_id] = (center_x, center_y, exit_buffer)
                next_id += 1
                total_in_count += 1  # 새 ID 감지 시 입장으로 간주

            current_ids.add(object_id)  # 현재 프레임에 감지된 ID 추가

            # 경계 상자와 ID 표시
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, f'ID: {object_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # 이전 프레임에서 존재했지만 현재 프레임에 없는 ID의 프레임 수 감소
    # 특정 사람이 다른 물체에 가려지거나 했을 떄도 계속 같은 사람임을 나타내주기 위한 코드
    for id in list(detected_ids.keys()):
        if id not in current_ids:
            prev_x, prev_y, frames = detected_ids[id]
            if frames > 0:
                detected_ids[id] = (prev_x, prev_y, frames - 1)
            else:
                del detected_ids[id]
                total_out_count += 1  # ID가 사라진 경우 퇴장으로 간주

    # 누적 카운트 표시
    cv2.putText(frame, f'Total In: {total_in_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'Total Out: {total_out_count}', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # 비디오에 현재 프레임 저장
    out.write(frame)

    # Colab 환경에서 현재 프레임 표시
    # cv2_imshow(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 모든 작업 완료 후 자원 해제
cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 6 persons, 2 cars, 1 handbag, 171.7ms
Speed: 5.7ms preprocess, 171.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 1 car, 143.9ms
Speed: 3.7ms preprocess, 143.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 2 cars, 1 handbag, 149.2ms
Speed: 3.6ms preprocess, 149.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1 car, 1 handbag, 171.3ms
Speed: 5.5ms preprocess, 171.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 2 cars, 1 handbag, 174.8ms
Speed: 3.8ms preprocess, 174.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 2 cars, 1 train, 2 handbags, 149.0ms
Speed: 3.7ms preprocess, 149.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 2 cars, 1 handbag, 153.7ms
Speed: 4.4ms preprocess, 153.7ms inference, 1.4ms postprocess per